In [15]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
import pandas as pd
import matplotlib.pyplot as plt
import glob
import os
import time
import numpy as np
import sys

In [17]:
ran_files = ['DlTxPhyStats.txt', 'UlRxPhyStats.txt', 
             'UlSinrStats.txt', 'UlPdcpStats.txt', 'DlRsrpSinrStats.txt', 
             'UlRlcStats.txt', 'UlMacStats.txt', 'UlTxPhyStats.txt', 
             'DlPdcpStats.txt', 'DlRlcStats.txt', 'DlRxPhyStats.txt', 
             'DlMacStats.txt']
app_files = ['httpClientRtt_trace.txt', 'delay_trace.txt', 'mpegPlayer_trace.txt', 
             'httpServerDelay_trace.txt', 'httpClientDelay_trace.txt', 'dashClient_trace.txt', 
             'rtt_trace.txt', 'mobility_trace.txt', 'flow_trace.txt']

files_with_bytes = ['DlTxPhyStats.txt', 'DlRxPhyStats.txt',
                    'UlTxPhyStats.txt', 'UlRxPhyStats.txt',
                    'DlMacStats.txt', 'UlMacStats.txt',
                    'DlRlcStats.txt', 'UlRlcStats.txt',  
                    'DlPdcpStats.txt', 'UlPdcpStats.txt']
files = ran_files + app_files # this has all files except interference 
not_to_be_used_now=['UlInterferenceStats.txt']# this is not in files




file_name_to_tstamp_unit = {
    'DlTxPhyStats.txt':'ms', 'UlRxPhyStats.txt':'ms', 
             'UlSinrStats.txt':'s', 'UlPdcpStats.txt':'s', 'DlRsrpSinrStats.txt':'s', 
             'UlRlcStats.txt':'s', 'UlMacStats.txt':'s', 'UlTxPhyStats.txt':'ms', 
             'DlPdcpStats.txt':'s', 'DlRlcStats.txt':'s', 'DlRxPhyStats.txt':'ms', 
             'DlMacStats.txt':'s'
}

files_with_trailing_tab=['DlRlcStats.txt', 'UlRlcStats.txt',  
                    'DlPdcpStats.txt', 'UlPdcpStats.txt']

files_that_use_upper_case_cellid=['UlPdcpStats.txt', 'UlRlcStats.txt', 'DlPdcpStats.txt', 'DlRlcStats.txt']
#temp_subset = ['delay_trace.txt', 'mobility_trace.txt', 'flow_trace.txt']
temp_subset = ['DlPdcpStats.txt', 'UlRxPhyStats.txt', 'delay_trace.txt', ]

data_dir = '../lte_scenario1/runs/'

In [18]:
total_num_cells=21
total_num_ues=630

In [19]:
ueIds = np.arange(0,total_num_ues)

isIncluded=np.array( [((x % 5) == 0) for x in range(0,total_num_ues)], dtype=bool)
trafficClass1_ueIds=ueIds[isIncluded]

isIncluded=np.array( [ ( ((x % 5) != 0) and (x%2 ==0) ) for x in range(0,total_num_ues)], dtype=bool)
trafficClass2_ueIds=ueIds[isIncluded]

isIncluded=np.array( [ ( ((x % 5) != 0) and (x%2 !=0) ) for x in range(0,total_num_ues)], dtype=bool)
trafficClass3_ueIds=ueIds[isIncluded]

print('expected num UEs with video streams:', len(trafficClass2_ueIds))
print('expected num UEs with web browsing:', len(trafficClass3_ueIds))
print('expected num UEs with UDP background traffic:', (len(trafficClass2_ueIds) + len(trafficClass3_ueIds)))
print('expected num UEs with delay measurements:', len(ueIds))
print('expected num UEs with only delay measurements and no background traffic:', len(trafficClass1_ueIds))

expected num UEs with video streams: 252
expected num UEs with web browsing: 252
expected num UEs with UDP background traffic: 504
expected num UEs with delay measurements: 630
expected num UEs with only delay measurements and no background traffic: 126


In [27]:
## sanity check 
##---------------------------------------

## iterate through all folders in run
#for run in os.listdir(data_dir):
for run in ['run19']:
    print (run)
    # list file names
    #print(os.listdir(data_dir+run))
    total_dl_tx = 0
    total_dl_rx = 0
    total_ul_tx = 0
    total_ul_rx = 0
    
    for file in app_files:
        print('--------------------------------------------')
        print(file)
        start_time = time.time()
        ## read file
        if file in files_with_trailing_tab:
            df = pd.read_csv(data_dir+run+'/'+file, sep='\t', usecols=range(0,18))
        else:
            df = pd.read_csv(data_dir+run+'/'+file, sep='\t')
            
        if file == 'rtt_trace.txt':
            # add an extra column name for reading
            names = ['tstamp_us', 'ueId', 'cellId', 'pktSize', 'seqNum', 'pktUid', 'txTstamp', 'txTstamp_copy', 'delay']
            df = pd.read_csv(data_dir+run+'/'+file, sep='\t', names=names, header=0)
            df.drop('txTstamp_copy', axis=1, inplace=True)
        if file == 'httpServerDelay_trace.txt':
            names = ['tstamp_us', 'ueId', 'cellId', 'delay']
            df = pd.read_csv(data_dir+run+'/'+file, sep='\t', names=names, skiprows=1)
            
            
        print('time to read file: ', (time.time() - start_time))
        ## print columns
        print(df.columns)
        ## do some file specific preprocessing
        ## change IMSI in internally generated logs to ueId
        if '% time' in df.columns:
            df.rename(columns = {'% time':'tstamp_us'}, inplace = True)
            if file_name_to_tstamp_unit[file] == 'ms':
                df['tstamp_us'] = df['tstamp_us']*(10**3)
            elif file_name_to_tstamp_unit[file] == 's':
                df['tstamp_us'] = df['tstamp_us']*(10**6)
                
        if '% start' in df.columns:
            # TO DO: check if this is actually micro seconds. I think it is seconds   
            df.rename(columns = {'% start':'tstamp_us'}, inplace = True)
            df.rename(columns = {'end':'end_timeslot_us'}, inplace = True)
            df['tstamp_us'] = df['tstamp_us']*(10**6)
            df['end_timeslot_us'] = df['end_timeslot_us']*(10**6)
            
        if 'IMSI' in df.columns:
            df['IMSI'] = df['IMSI'] - 1
            df.rename(columns = {'IMSI':'ueId'}, inplace = True)
        ## some internally generated logs use the naming 'CellId' replace that with 
        if 'CellId' in df.columns:
            df.rename(columns = {'CellId':'cellId'}, inplace = True)
        
        print(df.head())
        
        ## plot data frame against time
        #df.plot(x='tstamp_us', subplots=True, figsize=(20,10))
        #plt.show()
        
        ## display info about the UEs who have made entries in this file
        print('ueIds: min:', min(df['ueId'].value_counts().index), 'max:', max(df['ueId'].value_counts().index),
             'count:', len(df['ueId'].value_counts().index))
        ## display info about the Cells who have made entries in this file
        print('cellIds: min:', min(df['cellId'].value_counts().index), 'max:', max(df['cellId'].value_counts().index),
             'count:', len(df['cellId'].value_counts().index))
        ## total runtime of log
        print('log time (start, end): (', df['tstamp_us'].iloc[0]/(10**6), ', ' ,df['tstamp_us'].iloc[-1]/(10**6), ')')
        print('log runtime:', (df['tstamp_us'].iloc[-1] - df['tstamp_us'].iloc[0])/(10**6), ' seconds')
        
        # delay_trace flow_trace, and mobility_trace have run for the full duration 
        # only focus on these files for a bit
        ## separate data per UE 
        # 
        

run19
--------------------------------------------
httpClientRtt_trace.txt
time to read file:  0.029376983642578125
Index(['tstamp_us', 'ueId', 'cellId', 'pktSize', 'delay'], dtype='object')
   tstamp_us  ueId  cellId  pktSize   delay
0     592999   383       5    77739   71000
1     695999   267      15    54413  148000
2     716999    59      17    94344  157000
3     746999   511       7    98614  177000
4     772999    59      17     1005   24879
ueIds: min: 7 max: 609 count: 33
cellIds: min: 0 max: 20 count: 15
log time (start, end): ( 0.592999 ,  568.828999 )
log runtime: 568.236  seconds
--------------------------------------------
delay_trace.txt
time to read file:  2.7276875972747803
Index(['tstamp_us', 'dir', 'ueId', 'cellId', 'pktSize', 'seqNum', 'pktUid',
       'txTstamp', 'delay'],
      dtype='object')
   tstamp_us dir  ueId  cellId  pktSize  seqNum  pktUid  txTstamp  delay
0     504999  DL    24       3     1388       0    7436    501000   3999
1     504999  DL   308   